Importamos el modelo

In [1]:
import fasttext
import fasttext.util

#cargamos el modelo preentrenado en español
ft = fasttext.load_model('cc.es.300.bin')


funciones de análisis para encontrar palabras clave

In [2]:
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

# Descargar stop words
nltk.download('stopwords')
stop_words_es = set(stopwords.words('spanish'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words_es]
    return ' '.join(filtered_words)

def get_word_vectors_base(text, model):
    words = text.split()
    word_vectors = {word: model.get_word_vector(word) for word in words}
    return word_vectors

def get_word_vectors(text, model):
    words = text.split()
    word_vectors = {word: model.get_word_vector(word) for word in words if word not in stop_words_es}
    return word_vectors

#NO UTILIZA EL MODELO PREENTRENADO
#obtiene la importancia de las palabras en el texto basandose en la frecuencia de las palabras
def get_keywords_by_frequency(text, top_n=10):
    text = preprocess_text(text)
    words_base = text.split()
    word_scores = Counter(words_base)
    sorted_words = sorted(word_scores.items(), key=lambda kv: kv[1], reverse=True)
    return [word for word, score in sorted_words[:top_n]]

#obtiene la importancia de las palabras en el texto basandose en la norma de los vectores
def get_keywords_by_norm(text, model, top_n=10):
    text = preprocess_text(text)
    word_vectors = get_word_vectors(text, model)
    
    word_scores = {word: np.linalg.norm(vector) for word, vector in word_vectors.items()}
    sorted_words = sorted(word_scores.items(), key=lambda kv: kv[1], reverse=True)
    
    return [word for word, score in sorted_words[:top_n]]

#obtiene la importancia de las palabras en el texto basandose en la similitud coseno  
def get_keywords_by_similarity(text, model, top_n=10):
    text = preprocess_text(text)
    word_vectors = get_word_vectors(text, model)
    
    word_scores = {
        word: np.mean([
            np.dot(vector, other_vector) / (np.linalg.norm(vector) * np.linalg.norm(other_vector))
            for other_word, other_vector in word_vectors.items() if other_word != word
        ]) for word, vector in word_vectors.items()
    }
    
    sorted_words = sorted(word_scores.items(), key=lambda kv: kv[1], reverse=True)
    
    return [word for word, score in sorted_words[:top_n]]

#algunas palabras vacias no incluidad en las stop_words
palabras_vacias = ['asi','sino','gran']

def preprocess_text_tfidf(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    filtered_words = [word for word in words if (word not in stop_words_es and word not in palabras_vacias)]
    return ' '.join(filtered_words)

# Obtiene la importancia de las palabras en el texto basándose en el modelo TF-IDF
def get_keywords_by_tfidf(text, top_n=10):
    text = preprocess_text_tfidf(text)

     # Calcular la matriz TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text])
    
    # Obtener los índices de las palabras con mayor puntuación
    scores = tfidf_matrix.toarray().flatten()
    top_indices = np.argsort(scores)[::-1][:top_n]
    
     # Obtener las palabras correspondientes a los índices
    feature_names = vectorizer.get_feature_names_out()
    return [feature_names[i] for i in top_indices]



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mariajosevelazquez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pruebas y funciones auxiliares

In [3]:
def read_file_to_string(file_path):
    try:
        # Abrir el archivo en modo lectura
        with open(file_path, 'r', encoding='utf-8') as file:
            # Leer todo el contenido del archivo
            file_content = file.read()
        # Devolver el contenido como un string 
        return file_content
    except FileNotFoundError:
        print(f"Error: El archivo {file_path} no fue encontrado.")
        return ""
    except IOError:
        print(f"Error: No se pudo leer el archivo {file_path}.")
        return ""

In [4]:
#prueba texto 1

file_path = 'posts/prueba1.txt'
file_string = read_file_to_string(file_path)

sample_post = file_string
keywords1 = get_keywords_by_norm(sample_post, ft,10)
print("Palabras clave por norma:", keywords1)
#regresa none

keywords2 = get_keywords_by_similarity(sample_post, ft,10)
print("Palabras clave por similaridad:", keywords2)

keywords3 = get_keywords_by_frequency(sample_post,10)
print("Palabras clave por frecuencia:", keywords3)

keywords4 = get_keywords_by_tfidf(sample_post,10)
print("Palabras clave por TF-IDF:", keywords4)


Palabras clave por norma: ['mx', 'sr', '34', '32', 'tan', 'ven', 'mes', 'com', 'gota', 'daño']
Palabras clave por similaridad: ['solo', 'así', 'aun', 'ahora', 'efectivamente', 'tiempo', 'lleva', 'igual', 'casi', 'llega']
Palabras clave por frecuencia: ['fuga', 'agua', 'tejupilco', 'temascaltepec', 'potable', 'real', 'arriba', 'encuentra', 'mes', 'igual']
Palabras clave por TF-IDF: ['fuga', 'agua', 'tejupilco', 'temascaltepec', 'reparar', 'arriba', 'real', 'potable', 'igual', 'mes']


In [5]:
#prueba texto 1

file_path = 'posts/prueba2.txt'
file_string = read_file_to_string(file_path)

sample_post = file_string
keywords1 = get_keywords_by_norm(sample_post, ft,10)
print("Palabras clave por norma:", keywords1)
#regresa none

keywords2 = get_keywords_by_similarity(sample_post, ft,10)
print("Palabras clave por similaridad:", keywords2)

keywords3 = get_keywords_by_frequency(sample_post,10)
print("Palabras clave por frecuencia:", keywords3)

keywords4 = get_keywords_by_tfidf(sample_post,10)
print("Palabras clave por TF-IDF:", keywords4)

Palabras clave por norma: ['0', 'da', '25', 'si', '396', 'zit', 'dio', 'ser', 'vez', '2019']
Palabras clave por similaridad: ['denunciado', 'así', 'detenido', 'particular', 'informando', 'denuncia', 'presentado', 'ahora', 'detener', 'vez']
Palabras clave por frecuencia: ['uniformados', 'queja', 'policías', 'policía', 'michoacán', 'joven', 'libertad', 'empresa', 'cedh', 'estatales']
Palabras clave por TF-IDF: ['uniformados', 'queja', 'policías', 'joven', 'empresa', 'libertad', 'policía', 'michoacán', '1600', 'detenido']
